Jupyter Notebook For ReadMyMind, A CS 125 @ Illinois MP7 Project by Isaac Park and Mihir Pandya.

In [34]:
import tweepy
import pandas as pd
from textblob import TextBlob
from datetime import datetime

username = 'realDonaldTrump' # working example: Donald Trump

auth = tweepy.OAuthHandler('1X5fCqPl7yVvYxQjQJwkvavFD', 'NXbTDPP3HxlXOL5dWdCegEP09odLAkxUWlyRvZqXxtAtdX597G')
auth.set_access_token('925495606931546112-mn3Hda41LsZhbYAKJtddL7TulRKucuj', 'lvCFqSLv5YvOGzCINH6JZ5cBI1CEkPKrRioBn5Iuec3Tt')
api = tweepy.API(auth)
    
tweets_df = pd.DataFrame({
    'Timestamp': (),
    'Likes': (),
    'Retweets': (),
    'Text': (),
    'Sentences': (),
    'Keywords': ()
})

tweets_df = tweets_df[['Timestamp', 'Likes', 'Retweets', 'Text', 'Sentences', 'Keywords']]
    
recent_tweets = api.user_timeline(screen_name = username, count=100, tweet_mode="extended") # analyzing 100 tweets
for status in recent_tweets:
    test = status.full_text
    if test[:2] != 'RT': # removing retweets made by the user
        status_data = pd.Series([status.created_at, status.favorite_count, status.retweet_count, status.full_text], 
                                index=['Timestamp', 'Likes', 'Retweets', 'Text'])
    tweets_df = tweets_df.append(status_data, ignore_index = True)
    
tweets_df = tweets_df.drop_duplicates(subset='Text') # just in case, remove any duplicate tweets
tweets_df = tweets_df.astype('object')

for i in range(len(tweets_df)):
    content = tweets_df.iloc[i]['Text']
    if 'http' in content:
        j = content.index('http')
        content = content[:j] # cleaning text of the tweet by removing the link at the end and newline characters
    content = content.replace('\n', '')
    
    blob = TextBlob(content)
    sentiments = {}
    
    for sent in blob.sentences: # generating sentiment polarity values for each sentence in the tweet
        sentiments[str(sent)] = sent.sentiment.polarity
        
    tweets_df.iloc[i]['Sentences'] = sentiments # insert dictionary of sentence: sentiment value into dataframe
    
    tweets_df.iloc[i]['Timestamp'] = tweets_df.iloc[i]['Timestamp'].to_pydatetime() # convert pandas.tslib.Timestamp object to datetime
    
    # Keyword extraction goes here
    # append list of keywords

tweets_df

,Timestamp,Likes,Retweets,Text,Sentences,Keywords
0,2017-12-05 23:13:28,28541,7476,"Join me this Friday in Pensacola, Florida at t...","{'Join me this Friday in Pensacola, Florida at...",NaN
1,2017-12-05 17:46:45,49302,10770,"Yesterday, I was thrilled to be with so many W...","{'Yesterday, I was thrilled to be with so many...",NaN
2,2017-12-04 12:35:39,69182,17620,A must watch: Legal Scholar Alan Dershowitz wa...,{'A must watch: Legal Scholar Alan Dershowitz ...,NaN
3,2017-12-04 12:03:18,83690,15906,"With the great vote on Cutting Taxes, this cou...","{'With the great vote on Cutting Taxes, this c...",NaN
4,2017-12-04 12:00:08,88605,20926,Putting Pelosi/Schumer Liberal Puppet Jones in...,{'Putting Pelosi/Schumer Liberal Puppet Jones ...,NaN
5,2017-12-04 11:17:05,89941,24850,Democrats refusal to give even one vote for ma...,{'Democrats refusal to give even one vote for ...,NaN
6,2017-12-03 13:36:45,121778,37016,Report: “ANTI-TRUMP FBI AGENT LED CLINTON EMAI...,{'Report: “ANTI-TRUMP FBI AGENT LED CLINTON EM...,NaN
7,2017-12-03 13:15:21,119704,36588,People who lost money when the Stock Market we...,{'People who lost money when the Stock Market ...,NaN
8,2017-12-03 13:00:47,117300,28811,"After years of Comey, with the phony and disho...","{'After years of Comey, with the phony and dis...",NaN
9,2017-12-03 12:42:47,84502,25661,"Tainted (no, very dishonest?) FBI “agent’s rol...","{'Tainted (no, very dishonest?)': 0.1153846153...",NaN
